In [14]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import os, sys

from skimage.io import imread, imsave
from skimage.transform import rotate
from skimage.util import img_as_ubyte, img_as_float

from subprocess import check_output, call

In [53]:
stack = 'MD594'

img_dir = '/home/yuncong/CSHL_data_processed/'+stack+'_lossless_aligned_cropped_downscaled_stereotactic'
filenames = sorted(os.listdir(img_dir))
rotated_w, rotated_h = map(int, check_output("identify -format %%Wx%%H %s" % img_dir + '/' + filenames[1], shell=True).split('x'))

print rotated_w, rotated_h 

21728 14816


In [73]:
interaural_center_tb_allStacks = {'MD589': (51, 342), 'MD594': (138, 337)}
pitch_angle_allStacks = {'MD589': -18.5, 'MD594': 0} # negative for counter-clockise rotation, in degrees

interaural_center_tb = interaural_center_tb_allStacks[stack]
interaural_center_lossless = map(lambda x: x*32, interaural_center_tb)
pitch_angle = pitch_angle_allStacks[stack]

# interaural_center_lossless_allStacks = dict([(l, (pt[0]*32, pt[1]*32)) for l, pt in interaural_center_tb_allStacks.iteritems()])
# interaural_center_lossless = interaural_center_lossless_allStacks[stack]

In [74]:
from skimage.transform import SimilarityTransform, warp

def get_rotate_transform(shape, angle, resize=False, center=None, order=1, mode='constant',
           cval=0, clip=True, preserve_range=False):

    rows, cols = shape

    # rotation around center
    if center is None:
        center = np.array((cols, rows)) / 2. - 0.5
    else:
        center = np.asarray(center)
                
    tform1 = SimilarityTransform(translation=-center)
    tform2 = SimilarityTransform(rotation=np.deg2rad(angle))
    tform3 = SimilarityTransform(translation=center)
    tform = tform1 + tform2 + tform3

    output_shape = None
    if resize:
        # determine shape of output image
        corners = np.array([[1, 1], [1, rows], [cols, rows], [cols, 1]])
        corners = tform(corners - 1)
        minc = corners[:, 0].min()
        minr = corners[:, 1].min()
        maxc = corners[:, 0].max()
        maxr = corners[:, 1].max()
                
        out_rows = maxr - minr + 1
        out_cols = maxc - minc + 1
        output_shape = np.ceil((out_rows, out_cols))
        
        # fit output image in new shape
        translation = -np.asarray((minc, minr))
        tform4 = SimilarityTransform(translation=translation)
        tform = tform + tform4

    return tform, output_shape

In [75]:
tform, output_shape = get_rotate_transform((orig_h, orig_w), pitch_angle, resize=True, center=interaural_center_lossless, cval=0)
print output_shape

[ 17632.  16704.]


In [112]:
# interaural_center_tb_shifted = tform(interaural_center_tb)[0].astype(np.int)
# print interaural_center_tb_shifted

interaural_center_lossless_shifted = tform(interaural_center_lossless)[0].astype(np.int)
print interaural_center_lossless_shifted

grid_viz = np.zeros((rotated_h, rotated_w, 4), np.uint8)

# 15.2 um / pixel
micrometers_per_pixel = 15.2 / 32
micrometers_per_tick = 250
green_tick_interval_mm = 1000
lw = 8

tick_interval_pixel = micrometers_per_tick / micrometers_per_pixel
green_tick_interval_pixel = green_tick_interval_mm / micrometers_per_pixel

# gray grids are 100 um apart
for c in np.arange(interaural_center_lossless_shifted[0], 0, -tick_interval_pixel).astype(np.int):
    grid_viz[:, c-lw:c+lw, 3] = 255*.1

for c in np.arange(interaural_center_lossless_shifted[0], rotated_w, tick_interval_pixel).astype(np.int):
    grid_viz[:, c-lw:c+lw, 3] = 255*.1
    
for r in np.arange(interaural_center_lossless_shifted[1], 0, -tick_interval_pixel).astype(np.int):
    grid_viz[r-lw:r+lw, :, 3] = 255*.1

for r in np.arange(interaural_center_lossless_shifted[1], rotated_h, tick_interval_pixel).astype(np.int):
    grid_viz[r-lw:r+lw, :, 3] = 255*.1


# green grids are 1 mm apart

neg_xticks = np.arange(interaural_center_lossless_shifted[0], 0, -green_tick_interval_pixel).astype(np.int)
pos_xticks = np.arange(interaural_center_lossless_shifted[0], rotated_w, green_tick_interval_pixel).astype(np.int)
xticks = np.r_[neg_xticks[::-1][:-1], pos_xticks]

neg_yticks = np.arange(interaural_center_lossless_shifted[1], 0, -green_tick_interval_pixel).astype(np.int)
pos_yticks = np.arange(interaural_center_lossless_shifted[1], rotated_h, green_tick_interval_pixel).astype(np.int)
yticks = np.r_[neg_yticks[::-1][:-1], pos_yticks]
    
for c in xticks:
    grid_viz[:, c-lw:c+lw, :] = (0,255,0,255*.4)

for r in yticks:
    grid_viz[r-lw:r+lw, :, :] = (0,255,0,255*.4)

# red line defines origin (interaural)
cx, cy = interaural_center_lossless_shifted
grid_viz[:, cx-lw:cx+lw, :] = (255,0,0,255*.4)
grid_viz[cy-lw:cy+lw, :, :] = (255,0,0,255*.4)

n_pos_x = len(pos_xticks)
n_neg_x = len(neg_xticks)-1
n_pos_y = len(pos_yticks)
n_neg_y = len(neg_yticks)-1

# coordinate texts
for xi, x in enumerate(xticks):
    for yi, y in enumerate(yticks):
        cv2.putText(grid_viz, '(%d,%d)'%(n_neg_x-xi, n_neg_y-yi), (x,y-20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 3, ((0,255,0,255*.4)), 5)

[ 4416 10784]


In [113]:
import cv2
cv2.imwrite('/home/yuncong/CSHL_data_processed/'+stack+'_lossless_aligned_cropped_stereotacticGrids.png', 
            grid_viz[...,[2,1,0,3]]);

# imsave('/home/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_stereotacticGrids.png', grid_viz)

True

In [33]:
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import alpha_blending, display

In [237]:
zip(range(49, 115), np.linspace(2.64, 0, 114-49+1))

[(49, 2.6400000000000001),
 (50, 2.5993846153846154),
 (51, 2.5587692307692307),
 (52, 2.5181538461538464),
 (53, 2.4775384615384617),
 (54, 2.436923076923077),
 (55, 2.3963076923076922),
 (56, 2.355692307692308),
 (57, 2.3150769230769233),
 (58, 2.2744615384615385),
 (59, 2.2338461538461538),
 (60, 2.1932307692307695),
 (61, 2.1526153846153848),
 (62, 2.1120000000000001),
 (63, 2.0713846153846154),
 (64, 2.0307692307692307),
 (65, 1.9901538461538464),
 (66, 1.9495384615384617),
 (67, 1.9089230769230769),
 (68, 1.8683076923076924),
 (69, 1.8276923076923079),
 (70, 1.7870769230769232),
 (71, 1.7464615384615385),
 (72, 1.705846153846154),
 (73, 1.6652307692307693),
 (74, 1.6246153846153848),
 (75, 1.5840000000000001),
 (76, 1.5433846153846156),
 (77, 1.5027692307692309),
 (78, 1.4621538461538464),
 (79, 1.4215384615384616),
 (80, 1.3809230769230771),
 (81, 1.3403076923076924),
 (82, 1.2996923076923077),
 (83, 1.2590769230769232),
 (84, 1.2184615384615385),
 (85, 1.177846153846154),
 (86,

In [6]:
for f in filenames:
    print f

MD589_0068_thumbnail_aligned_cropped_stereotactic.tif
MD589_0119_thumbnail_aligned_cropped_stereotactic.tif
MD589_0131_thumbnail_aligned_cropped_stereotactic.tif
MD589_0054_thumbnail_aligned_cropped_stereotactic.tif
MD589_0101_thumbnail_aligned_cropped_stereotactic.tif
MD589_0112_thumbnail_aligned_cropped_stereotactic.tif
MD589_0090_thumbnail_aligned_cropped_stereotactic.tif
MD589_0130_thumbnail_aligned_cropped_stereotactic.tif
MD589_0140_thumbnail_aligned_cropped_stereotactic.tif
MD589_0110_thumbnail_aligned_cropped_stereotactic.tif
MD589_0093_thumbnail_aligned_cropped_stereotactic.tif
MD589_0105_thumbnail_aligned_cropped_stereotactic.tif
MD589_0132_thumbnail_aligned_cropped_stereotactic.tif
MD589_0073_thumbnail_aligned_cropped_stereotactic.tif
MD589_0108_thumbnail_aligned_cropped_stereotactic.tif
MD589_0152_thumbnail_aligned_cropped_stereotactic.tif
MD589_0094_thumbnail_aligned_cropped_stereotactic.tif
MD589_0107_thumbnail_aligned_cropped_stereotactic.tif
MD589_0061_thumbnail_aligned